In [1]:
!pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import duckdb
import pandas as pd
from teachdb.teachdb import connect_db
# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1)
con = duckdb.connect(":memory:")

In [3]:
#team_con = duckdb.connect(":memory:") add at end of data loading

In [4]:
# Load data
pipeline_df = pd.read_csv('//content//drive//MyDrive//Data Analysis Projects//Data Projects//CRMSalesOpportunities//sales_pipeline.csv')
team_df = pd.read_csv('//content//drive//MyDrive//Data Analysis Projects//Data Projects//CRMSalesOpportunities//sales_teams.csv')
account_df = pd.read_csv('//content//drive//MyDrive//Data Analysis Projects//Data Projects//CRMSalesOpportunities//accounts.csv')
products_df = pd.read_csv('//content//drive//MyDrive//Data Analysis Projects//Data Projects//CRMSalesOpportunities//products.csv')

data_df = {"sales_pipeline" : pipeline_df, "sales_teams" : team_df, "accounts" : account_df, "products" : products_df}

In [5]:
connect_db(con, data_df)
%sql con

In [6]:
%%sql
SELECT *
FROM sales_pipeline
LIMIT 3

,opportunity_id,sales_agent,product,account,deal_stage,engage_date,close_date,close_value
0,1C1I7A6R,Moses Frase,GTX Plus Basic,Cancity,Won,2016-10-20,2017-03-01,1054.0
1,Z063OYW0,Darcel Schlecht,GTXPro,Isdom,Won,2016-10-25,2017-03-11,4514.0
2,EC4QE1BX,Darcel Schlecht,MG Special,Cancity,Won,2016-10-25,2017-03-07,50.0


In [7]:
%%sql
SELECT *
FROM sales_teams
LIMIT 3

,sales_agent,manager,regional_office
0,Anna Snelling,Dustin Brinkmann,Central
1,Cecily Lampkin,Dustin Brinkmann,Central
2,Versie Hillebrand,Dustin Brinkmann,Central


In [8]:
%%sql
SELECT *
FROM accounts
LIMIT 3

,account,sector,year_established,revenue,employees,office_location,subsidiary_of
0,Acme Corporation,technolgy,1996,1100.04,2822,United States,None
1,Betasoloin,medical,1999,251.41,495,United States,None
2,Betatech,medical,1986,647.18,1185,Kenya,None


In [9]:
%%sql
SELECT *
FROM products

,product,series,sales_price
0,GTX Basic,GTX,550
1,GTX Pro,GTX,4821
2,MG Special,MG,55
3,MG Advanced,MG,3393
4,GTX Plus Pro,GTX,5482
5,GTX Plus Basic,GTX,1096
6,GTK 500,GTK,26768


#Data Cleanup Tasks






##General
* Missing/invalid values
* Misspellings
* Duplicates
* Outliers
* Consistent datatypes
* readability (column names, consistent cases)

In [10]:
%%sql
SELECT table_name, column_name, data_type
FROM information_schema.columns
ORDER BY table_name, column_name

,table_name,column_name,data_type
0,accounts,account,VARCHAR
1,accounts,employees,BIGINT
2,accounts,office_location,VARCHAR
3,accounts,revenue,DOUBLE
4,accounts,sector,VARCHAR
5,accounts,subsidiary_of,VARCHAR
6,accounts,year_established,BIGINT
7,products,product,VARCHAR
8,products,sales_price,BIGINT
9,products,series,VARCHAR


##sales_pipeline
* Blanks in account column
* Blanks in close_date and close_value tied to “Engaging” value in deal_stage
* 0s in close_value tied to “Lost” value in deal_stage
* close_date not before engage_date
* Sales agents found in sales_team
* Accounts found on accounts table

In [11]:
%%sql
SELECT DISTINCT sales_agent
FROM sales_pipeline
ORDER BY sales_agent

,sales_agent
0,Anna Snelling
1,Boris Faz
2,Cassey Cress
3,Cecily Lampkin
4,Corliss Cosme
5,Daniell Hammack
6,Darcel Schlecht
7,Donn Cantrell
8,Elease Gluck
9,Garret Kinder


In [12]:
%%sql
SELECT sales_agent
FROM sales_pipeline
WHERE sales_agent NOT IN (
  SELECT P.sales_agent
  FROM sales_pipeline AS P INNER JOIN sales_teams AS T
    ON P.sales_agent = T.sales_agent
)

,sales_agent


In [13]:
%%sql
SELECT account, close_date, close_value, deal_stage, engage_date, opportunity_id, product, sales_agent, COUNT(*) AS count
FROM sales_pipeline
GROUP BY account, close_date, close_value, deal_stage, engage_date, opportunity_id, product, sales_agent
HAVING COUNT(*) > 1;

,account,close_date,close_value,deal_stage,engage_date,opportunity_id,product,sales_agent,count


In [14]:
#check for missing values on closed or lost deal stages
%%sql
select deal_stage, COUNT(deal_stage)
from sales_pipeline
where
  close_date is null
GROUP BY deal_stage

,deal_stage,count(deal_stage)
0,Engaging,1589
1,Prospecting,500


In [15]:
%%sql
SELECT deal_stage, COUNT(deal_stage)
FROM sales_pipeline
where
  close_value is null
GROUP BY deal_stage

,deal_stage,count(deal_stage)
0,Engaging,1589
1,Prospecting,500


In [16]:
%%sql
SELECT account
FROM sales_pipeline
WHERE account NOT IN (
  SELECT S.account
  FROM sales_pipeline AS S INNER JOIN accounts AS A
ON S.account = A.account
)



,account


In [17]:
%%sql
-- Add a new column with DATE datatype
ALTER TABLE sales_pipeline ADD COLUMN engage_date_new DATE;

-- Update the new column with converted values from the old column
UPDATE sales_pipeline
SET engage_date_new = CAST(engage_date AS date); -- This assumes engage_date is in 'YYYY-MM-DD' format

-- Drop the old column
ALTER TABLE sales_pipeline DROP COLUMN engage_date;

-- Rename the new column to the original column name
ALTER TABLE sales_pipeline RENAME COLUMN engage_date_new TO engage_date;

,Success


In [18]:
%%sql
-- Add a new column with DATE datatype
ALTER TABLE sales_pipeline ADD COLUMN close_date_new DATE;

-- Update the new column with converted values from the old column
UPDATE sales_pipeline
SET close_date_new = CAST(close_date AS date); -- This assumes close_date is in 'YYYY-MM-DD' format

-- Drop the old column
ALTER TABLE sales_pipeline DROP COLUMN close_date;

-- Rename the new column to the original column name
ALTER TABLE sales_pipeline RENAME COLUMN close_date_new TO close_date;

,Success


In [19]:
%%sql
-- One-hot encoding deal stage
ALTER TABLE sales_pipeline ADD COLUMN Prospecting INT;
ALTER TABLE sales_pipeline ADD COLUMN Engaging INT;
ALTER TABLE sales_pipeline ADD COLUMN Won INT;
ALTER TABLE sales_pipeline ADD COLUMN Lost INT;


UPDATE sales_pipeline SET Prospecting = 1, Engaging = 0, Won = 0, Lost = 0 WHERE deal_stage = 'Prospecting';
UPDATE sales_pipeline SET Prospecting = 1, Engaging = 1, Won = 0, Lost = 0 WHERE deal_stage = 'Engaging';
UPDATE sales_pipeline SET Prospecting = 1, Engaging = 1, Won = 1, Lost = 0 WHERE deal_stage = 'Won';
UPDATE sales_pipeline SET Prospecting = 1, Engaging = 1, Won = 0, Lost = 1 WHERE deal_stage = 'Lost';

,Count
0,2473


In [20]:
%%sql
SELECT table_name, column_name, data_type
FROM information_schema.columns
ORDER BY table_name, column_name

,table_name,column_name,data_type
0,accounts,account,VARCHAR
1,accounts,employees,BIGINT
2,accounts,office_location,VARCHAR
3,accounts,revenue,DOUBLE
4,accounts,sector,VARCHAR
5,accounts,subsidiary_of,VARCHAR
6,accounts,year_established,BIGINT
7,products,product,VARCHAR
8,products,sales_price,BIGINT
9,products,series,VARCHAR


In [21]:
%%sql

SELECT opportunity_id, product
FROM sales_pipeline
WHERE opportunity_id NOT IN (SELECT S.opportunity_id
FROM sales_pipeline AS S
INNER JOIN products AS P ON S.product = P.product
)
LIMIT 5
-- Mispellings in product column

,opportunity_id,product
0,Z063OYW0,GTXPro
1,KWVA7VR1,GTXPro
2,902REDPA,GTXPro
3,KNY1OSAB,GTXPro
4,JYKM0B00,GTXPro


The product GTX Pro is missing a space in the sales_pipeline table, resulting in incosistencies between tables

In [22]:
%%sql

UPDATE sales_pipeline
SET product = 'GTX Pro'
WHERE product = 'GTXPro'

,Count
0,1480


In [23]:
%%sql

SELECT deal_stage, COUNT(deal_stage)
FROM sales_pipeline
WHERE account IS NULL
GROUP BY deal_stage

,deal_stage,count(deal_stage)
0,Engaging,1088
1,Prospecting,337


In [24]:
%%sql

SELECT sales_agent, COUNT(sales_agent)
FROM sales_pipeline
WHERE account IS NULL
GROUP BY sales_agent

,sales_agent,count(sales_agent)
0,Cassey Cress,56
1,Elease Gluck,33
2,Zane Levy,59
3,Daniell Hammack,47
4,Rosalina Dieter,34
5,Corliss Cosme,54
6,Vicki Laflamme,77
7,Maureen Marcano,56
8,Violet Mclelland,48
9,Rosie Papadopoulos,24


##accounts
* year_established values consistent for years
* revenue values make sense


In [25]:
%%sql

SELECT MIN(year_established), MAX (year_established)
FROM accounts


,min(year_established),max(year_established)
0,1979,2017


In [26]:
%%sql

SELECT MIN(revenue), MAX(revenue), MEAN(revenue)
FROM accounts

,min(revenue),max(revenue),mean(revenue)
0,4.54,11698.03,1994.632941


Research needed to see if values are in millions or other denomination (ie. hundred thousands)

In [27]:
%%sql
COPY sales_pipeline TO 'sales_pipeline_cleaned.csv' (HEADER, DELIMITER ',');

,Count
0,8800
